tout ensemble bien propre, + goodies: tensorboard, load/save


In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Processing...
Done!


In [14]:
class MyModel(nn.Module):
    """modèle basique linéaire"""
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(28*28, 10)
            
    
    def forward(self,input):
        scores = self.linear(self.flatten(input))
        return scores

In [15]:
model = MyModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # informe le modèle qu'il est en mode "train" (utile pour certaines architectures)
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader)):
        # on transfère sur CPU/GPU
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # la version simple pour suivre l'expé; cf tensorboard plus loin pour une méthode plus pratique en pratique
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


21it [00:00, 205.97it/s]

loss: 2.367205  [    0/60000]


133it [00:00, 218.11it/s]

loss: 2.156315  [ 6400/60000]


...

loss: 0.933975  [51200/60000]


938it [00:04, 224.65it/s]


loss: 0.840832  [57600/60000]


100%|██████████| 157/157 [00:00<00:00, 243.02it/s]

Test Error: 
 Accuracy: 6.6%, Avg loss: 2.679507 

Done!


In [19]:
#torch.save(model.state_dict(), "model.pth")

In [ ]:
# todo: mettre dans la fonction de train, avec log=writer
# mieux : en sauvant pour visualisation dans tensorboard
# need to run in terminal: tensorboard --logdir=runs
# and open interface http://localhost:6006/
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

model = MyModel()
for epoch in range(10):
    print(f"epoch={epoch}")
    for input, target in tqdm(train_dataloader):    
        optimizer.zero_grad() 
        output = model(input)
        loss = loss_fn(output, target) 
        writer.add_scalar('Loss/train', loss, epoch)
        loss.backward() 
        optimizer.step() 